# 퀴즈 문제
LeNet-5 모델을 이용하여 MNIST 데이터 셋의 정확도를 측정하시오.

인공지능 수업 - 최유경 교수님

### [변경하지 말것] 데이터 로더 
- 입력 영상 데이터 mnist 28x28 ==> 32x32 로 가공하여 제공함

In [1]:

import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
from tensorflow.contrib.layers import flatten
from skimage import transform


def resize_batch(imgs):
    # A function to resize a batch of MNIST images to (32, 32)
    # Args:
    #   imgs: a numpy array of size [batch_size, 28 X 28].
    # Returns:
    #   a numpy array of size [batch_size, 32, 32].
    imgs = imgs.reshape((-1, 28, 28, 1))
    resized_imgs = np.zeros((imgs.shape[0], 32, 32, 1))
    for i in range(imgs.shape[0]):
        resized_imgs[i, ..., 0] = transform.resize(imgs[i, ..., 0], (32, 32))
    return resized_imgs





tf.set_random_seed(777)  # reproducibility
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


'''
[데이터 참고] 
mnist.train.num_examples
mnist.train.next_batch
mnist.train.images
mnist.train.labels
mnist.test.images
mnist.test.labels
'''

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


'\n[데이터 참고] \nmnist.train.num_examples\nmnist.train.next_batch\nmnist.train.images\nmnist.train.labels\nmnist.test.images\nmnist.test.labels\n'

### [작성필요] 모델을 작성하세요.


![대체 텍스트](https://github.com/wingdi/Lenet-5/raw/master/src/imgs/lenet-5.png)

In [0]:

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# input place holders
X = tf.placeholder(tf.float32, (None,32,32,1))
Y = tf.placeholder(tf.float32, [None, 10])


In [3]:
test = resize_batch(mnist.train.images[0:20])

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [6]:
# 디버깅, 기본에 충실하자. 더러운 코드여도 실행되면 일단 쓰고 최적화 하자.
# LeNet5 Model

def LeNet5(input_x):
  mu = 0
  sigma = 0.1


  conv1_w = tf.Variable(tf.truncated_normal([5,5,1,6],mean=mu, stddev=sigma))
  conv1 = tf.nn.conv2d(X,conv1_w,strides=[1,1,1,1],padding='VALID')
  conv1_relu = tf.nn.relu(conv1)

  pool1 = tf.nn.max_pool(conv1_relu,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')

  conv2_w = tf.Variable(tf.truncated_normal([5,5,6,16],mean=mu,stddev=sigma))
  conv2 = tf.nn.conv2d(pool1,conv2_w,strides=[1,1,1,1],padding='VALID')
  conv2_relu = tf.nn.relu(conv2)

  pool2 = tf.nn.max_pool(conv2_relu,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')

  pool2_flat = tf.reshape(pool2,shape=[-1,5*5*16])

  fc1_w = tf.Variable(tf.truncated_normal(shape=[5*5*16,120],mean=mu,stddev=sigma))
  fc1_b = tf.Variable(tf.zeros(120))
  fc1 = tf.matmul(pool2_flat,fc1_w)+fc1_b
  fc1_relu = tf.nn.relu(fc1)

  fc2_w = tf.Variable(tf.random_normal(shape=[120,84],mean=mu,stddev=sigma))
  fc2_b = tf.Variable(tf.zeros(84))
  fc2 = tf.matmul(fc1_relu,fc2_w)+fc2_b
  fc2_relu = tf.nn.relu(fc2)

  fc3_w = tf.Variable(tf.random_normal(shape=[84,10],mean=mu,stddev=sigma))
  fc3_b = tf.Variable(tf.zeros(10))
  logits = tf.matmul(fc2_relu,fc3_w)+fc3_b
  
  return logits

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

logits = LeNet5(X)

sess= tf.Session()
sess.run(tf.global_variables_initializer())
val = logits.eval(session=sess,feed_dict={X:test})
print(logits.shape)



(?, 10)


In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('learning start!')

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples/ batch_size)
  
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = resize_batch(batch_xs)
    
    c,_ = sess.run([cost,optimizer],feed_dict={X:batch_xs,Y:batch_ys,keep_prob:0.7})
    avg_cost +=c/total_batch
    
  print('Epoch : {} , cost : {} '.format(epoch+1,avg_cost))
  
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:',sess.run(accuracy, feed_dict={X:resize_batch(mnist.test.images),Y:mnist.test.labels,keep_prob:1}))

r = random.randint(0,mnist.test.num_examples-1)

print('Label :',sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print('Prediction :',sess.run(tf.argmax(logits,1),feed_dict={X:resize_batch(mnist.test.images[r:r+1]),keep_prob:1}))

learning start!


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Epoch : 1 , cost : 0.29467631243846626 
